# Re-Sampling of OPM Data and saving it to a .fif file

In [1]:
#Import necessary libraries and modules
import mne
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import array


#### Checking if the files are accessible

In [3]:
# List contents of the directory
directory_path = '/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings'
files = os.listdir(directory_path)
file_name='Rec1.fif'
data_path=os.path.join(directory_path,file_name)
# Check if the specific file exists
file_exists = os.path.isfile(data_path)
print(f"File exists: {file_exists}")

File exists: True


#### Data Loading

In [4]:
#Data Loading
raw = mne.io.read_raw_fif(data_path, preload=True)
# Detailed information about the data
print(raw.info)

Opening raw data file /home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings/Rec1.fif...


/tmp/ipykernel_24561/1999083531.py:2: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings/Rec1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(data_path, preload=True)


Isotrak not found
    Range : 0 ... 1499999 =      0.000 ...   300.000 secs
Ready.
Reading 0 ... 1499999  =      0.000 ...   300.000 secs...
<Info | 16 non-empty values
 bads: []
 ch_names: s1_bz, L209_bz-s65, s2_bz, L207_bz-s66, R204_bz-s3, L205_bz-s67, ...
 chs: 72 Magnetometers, 1 Stimulus
 custom_ref_applied: False
 description: {"chassis":{"version":"0.7.7- ...
 experimenter: AR
 file_id: 4 items (dict)
 gantry_angle: 0.0
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 500.0 Hz
 meas_date: 2024-08-23 15:22:40 UTC
 meas_id: 4 items (dict)
 nchan: 73
 proj_id: 1 item (ndarray)
 proj_name: NoiseDetection
 projs: []
 sfreq: 5000.0 Hz
 xplotter_layout: None
>


### Filtering and Decimation

In [5]:
### Filtering and decimation
## Make sure Transition bandwidth is less than half of the upper cut-off.
current_sfreq = raw.info['sfreq']
desired_sfreq = 1000  # Hz
decim = np.round(current_sfreq / desired_sfreq).astype(int)
obtained_sfreq = current_sfreq / decim
lowpass_freq = obtained_sfreq / 4.

raw_filtered = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 265 samples (0.053 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    2.4s


In [ ]:
data_f=raw_filtered.get_data()
decimated_data_new = np.zeros((data_f.shape[0], (data_f.shape[1]-1) // ds_factor)) 

In [ ]:
for i, channel_data in enumerate(data_f):
    # decimation using scipy's decimate function
    manual_decimation = channel_data[:-ds_factor:ds_factor]
    #np.append(manual_decimation, 0)
    decimated_data_new[i,:]=manual_decimation

In [18]:
import mne
import os
import numpy as np
from scipy.signal import decimate

def resample_meg_data(fif_file_path, desired_sfreq=1000, output_file_path=None):
    """
    Function to resample MEG data from an original .fif file to a specified sampling frequency manually.

    Parameters:
    fif_file_path (str): Path to the original .fif file.
    desired_sfreq (int): Desired sampling frequency in Hz. Default is 1000 Hz.
    output_file_path (str): Optional output path for the new .fif file with resampled data.

    Returns:
    str: Path to the resampled .fif file.
    """
    # Check if the specific file exists
    if not os.path.isfile(fif_file_path):
        raise FileNotFoundError(f"The file {fif_file_path} does not exist.")
    
    # Load the raw data from the .fif file
    raw = mne.io.read_raw_fif(fif_file_path, preload=True)
    
    # Get current sampling frequency
    current_sfreq = raw.info['sfreq']
    
    if current_sfreq == desired_sfreq:
        print(f"The data is already at the desired sampling frequency of {desired_sfreq} Hz.")
        return fif_file_path
    
    # Compute decimation factor
    decim_factor = int(np.round(current_sfreq / desired_sfreq))
    obtained_sfreq = current_sfreq / decim_factor
    lowpass_freq = obtained_sfreq / 2.0
    
    # Apply low-pass filter to avoid aliasing
    raw_filtered = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)
    
    # Get the filtered data
    data_f = raw_filtered.get_data()
    
    # Decimate the data manually for each channel
    decimated_data_new = np.zeros((data_f.shape[0], data_f.shape[1] // decim_factor))
    
    for i, channel_data in enumerate(data_f):
        decimated_data_new[i, :] = decimate(channel_data, decim_factor, ftype='fir', zero_phase=True)
    
    # Create a new RawArray with the decimated data and correct info
    info_resampled = raw_filtered.info.copy()
    info_resampled['sfreq'] = obtained_sfreq  # Set the new sampling frequency indirectly by recalculating the time axis
    
    # Create a new MNE RawArray
    raw_resampled = mne.io.RawArray(decimated_data_new, info_resampled)
    
    # Set the output file path
    if output_file_path is None:
        output_file_path = fif_file_path.replace('.fif', f'_resampled_{desired_sfreq}Hz.fif')

    # Save the resampled data to a new .fif file
    raw_resampled.save(output_file_path, overwrite=True)
    
    return output_file_path


In [19]:
# Example usage
directory_path = '/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings'
file_name = 'RecN.fif'
data_path = os.path.join(directory_path, file_name)

output_file = resample_meg_data(data_path, desired_sfreq=1000)
print(f"Resampled data saved to: {output_file}")

Opening raw data file /home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings/RecN.fif...


/tmp/ipykernel_24561/1188707736.py:23: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings/RecN.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file_path, preload=True)


Isotrak not found
    Range : 0 ... 1499999 =      0.000 ...   300.000 secs
Ready.
Reading 0 ... 1499999  =      0.000 ...   300.000 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 500.00 Hz
- Upper transition bandwidth: 125.00 Hz (-6 dB cutoff frequency: 562.50 Hz)
- Filter length: 133 samples (0.027 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    3.9s


RuntimeError: sfreq cannot be set directly. Please use method inst.resample() instead.

In [15]:
# List contents of the directory
directory_path = '/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings'
files = os.listdir(directory_path)
file_name='RecN_resampled_1000Hz.fif'
data_path=os.path.join(directory_path,file_name)
# Check if the specific file exists
file_exists = os.path.isfile(data_path)
print(f"File exists: {file_exists}")

File exists: True


In [16]:
#Data Loading
raw = mne.io.read_raw_fif(data_path, preload=True)
# Detailed information about the data
print(raw.info)

Opening raw data file /home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings/RecN_resampled_1000Hz.fif...


/tmp/ipykernel_24561/1999083531.py:2: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings/RecN_resampled_1000Hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(data_path, preload=True)


Isotrak not found
    Range : 0 ... 299999 =      0.000 ...    60.000 secs
Ready.
Reading 0 ... 299999  =      0.000 ...    60.000 secs...
<Info | 16 non-empty values
 bads: []
 ch_names: s1_bz, L209_bz-s65, s2_bz, L207_bz-s66, R204_bz-s3, L205_bz-s67, ...
 chs: 72 Magnetometers, 1 Stimulus
 custom_ref_applied: False
 description: {"chassis":{"version":"0.7.7- ...
 experimenter: AR
 file_id: 4 items (dict)
 gantry_angle: 0.0
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 500.0 Hz
 meas_date: 2024-08-23 15:22:40 UTC
 meas_id: 4 items (dict)
 nchan: 73
 proj_id: 1 item (ndarray)
 proj_name: NoiseDetection
 projs: []
 sfreq: 5000.0 Hz
 xplotter_layout: None
>


In [17]:
raw.info['chs']

[{'scanno': 1,
  'logno': 0,
  'kind': 1 (FIFFV_MEG_CH),
  'range': 1.0,
  'cal': -1.8749804853390945e-15,
  'coil_type': 8101 (FIFFV_COIL_FIELDLINE_OPM_MAG_GEN1),
  'loc': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'unit': 112 (FIFF_UNIT_T),
  'unit_mul': 0 (FIFF_UNITM_NONE),
  'ch_name': 's1_bz',
  'coord_frame': 1 (FIFFV_COORD_DEVICE)},
 {'scanno': 2,
  'logno': 0,
  'kind': 1 (FIFFV_MEG_CH),
  'range': 1.0,
  'cal': -1.8749804853390945e-15,
  'coil_type': 8101 (FIFFV_COIL_FIELDLINE_OPM_MAG_GEN1),
  'loc': array([-0.0286461 , -0.04328817,  0.06105814, -0.86417657,  0.24221845,
         -0.4402943 ,  0.00289847, -0.87349087, -0.48622116, -0.50319999,
         -0.42160001,  0.75440001]),
  'unit': 112 (FIFF_UNIT_T),
  'unit_mul': 0 (FIFF_UNITM_NONE),
  'ch_name': 'L209_bz-s65',
  'coord_frame': 1 (FIFFV_COORD_DEVICE)},
 {'scanno': 3,
  'logno': 0,
  'kind': 1 (FIFFV_MEG_CH),
  'range': 1.0,
  'cal': -1.8749804853390945e-15,
  'coil_type': 8101 (FIFFV_COIL_FIELDLINE_OPM